In [ ]:
import $ivy.`org.apache.spark::spark-sql:3.5.7`
import org.apache.log4j.{Level, Logger}
Logger.getLogger("org").setLevel(Level.WARN)
import org.apache.spark._

import $ivy.$
import org.apache.log4j.{Level, Logger}
import org.apache.spark._

In [ ]:
val conf = new SparkConf().setAppName("RDD").setMaster("local[*]").set("spark.log.level", "WARN")
val sc = new SparkContext(conf)

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
26/02/08 19:34:07 INFO SparkContext: Running Spark version 3.5.7
26/02/08 19:34:07 INFO SparkContext: OS info Linux, 6.8.0-90-generic, amd64
26/02/08 19:34:07 INFO SparkContext: Java version 1.8.0_442
26/02/08 19:34:08 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting Spark log level to "WARN".


conf: SparkConf = org.apache.spark.SparkConf@d4223e6
sc: SparkContext = org.apache.spark.SparkContext@78a17c62

In [ ]:
val tripRaw = sc.textFile("data/tripdata.csv")
val taxiRaw = sc.textFile("data/taxi_zone_lookup.csv")

val tripHeader = tripRaw.first()
val taxiHeader = taxiRaw.first()

val trip = tripRaw.filter(_ != tripHeader)
val taxi = taxiRaw.filter(_ != taxiHeader)

tripRaw: rdd.RDD[String] = data/tripdata.csv MapPartitionsRDD[1] at textFile at cmd3.sc:1
taxiRaw: rdd.RDD[String] = data/taxi_zone_lookup.csv MapPartitionsRDD[3] at textFile at cmd3.sc:2
tripHeader: String = "VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee"
taxiHeader: String = "\"LocationID\",\"Borough\",\"Zone\",\"service_zone\""
trip: rdd.RDD[String] = MapPartitionsRDD[4] at filter at cmd3.sc:7
taxi: rdd.RDD[String] = MapPartitionsRDD[5] at filter at cmd3.sc:8

In [ ]:
trip.take(5).foreach(row => println(row))


2,2024-01-01T00:57:55.000,2024-01-01T01:17:43.000,1,1.72,1,N,186,79,2,17.7,1.0,0.5,0.0,0.0,1.0,22.7,2.5,0.0
1,2024-01-01T00:03:00.000,2024-01-01T00:09:36.000,1,1.8,1,N,140,236,1,10.0,3.5,0.5,3.75,0.0,1.0,18.75,2.5,0.0
1,2024-01-01T00:17:06.000,2024-01-01T00:35:01.000,1,4.7,1,N,236,79,1,23.3,3.5,0.5,3.0,0.0,1.0,31.3,2.5,0.0
1,2024-01-01T00:36:38.000,2024-01-01T00:44:56.000,1,1.4,1,N,79,211,1,10.0,3.5,0.5,2.0,0.0,1.0,17.0,2.5,0.0
1,2024-01-01T00:46:51.000,2024-01-01T00:52:57.000,1,0.8,1,N,211,148,1,7.9,3.5,0.5,3.2,0.0,1.0,16.1,2.5,0.0


In [ ]:
val tripParsed = trip.map { line =>
  val arr = line.split(",")
  try {
    val hour = arr(1).substring(11, 13).toInt
    val puLocationId = arr(7).toInt
    (puLocationId, hour)
  } catch {
    case _: Throwable => (-1, -1)  // на случай кривых строк
  }
}.filter { case (pu, h) => pu != -1 && h != -1 }

tripParsed: rdd.RDD[(Int, Int)] = MapPartitionsRDD[15] at filter at cmd11.sc:10

In [ ]:
tripParsed.take(5).foreach(row => println(row))

(186,0)
(140,0)
(236,0)
(79,0)
(211,0)


In [ ]:

val taxiParsed = taxi.map { line =>
  val arr = line.split(",")
  (arr(0).toInt, arr(1))
}

taxiParsed: rdd.RDD[(Int, String)] = MapPartitionsRDD[16] at map at cmd13.sc:2

In [ ]:

val joined = tripParsed.join(taxiParsed)

val ordersByBoroughHour = joined
  .map { case (_, (hour, borough)) => ((borough, hour), 1) }
  .reduceByKey(_ + _)

val result = ordersByBoroughHour.map {
  case ((borough, hour), count) => s"$borough,$hour,$count"
}


joined: rdd.RDD[(Int, (Int, String))] = MapPartitionsRDD[19] at join at cmd14.sc:2
ordersByBoroughHour: rdd.RDD[((String, Int), Int)] = ShuffledRDD[21] at reduceByKey at cmd14.sc:6
result: rdd.RDD[String] = MapPartitionsRDD[22] at map at cmd14.sc:8

In [ ]:
result.take(5).foreach(row1 => println(row1))

"EWR",8,3
"Unknown",16,122
"Manhattan",18,26960
"Manhattan",22,15612
"N/A",4,12


In [ ]:
result.saveAsTextFile("/tmp/orders_by_borough_hour")